<a href="https://colab.research.google.com/github/siva-Parvathi-M/E-Commerce-Sales-Returns/blob/main/E_Commerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load Data**

In [ ]:
import pandas as pd
from google.colab import files

# Replace 'your_file.csv' with your actual file name after uploading
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Projects/E-Commerce.csv", encoding='latin1')  # if it's CSV
# OR if it's Excel
# df = pd.read_excel("your_file.xlsx")

# Show first 5 rows
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [ ]:
# See column names and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [ ]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


**Data Cleaning & Preparation**

In [ ]:
# Check missing values
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


In [ ]:
# Drop rows with missing CustomerID
df = df.dropna(subset=['CustomerID'])

**Remove Returns & Cancellations**

In [ ]:
# Create a separate dataframe for returns
returns_df = df[df['Quantity'] < 0]

# Keep only valid sales (Quantity > 0)
df = df[df['Quantity'] > 0]

**Create New Columns**

In [ ]:
# Create TotalSales column
df['TotalSales'] = df['Quantity'] * df['UnitPrice']

# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Extract Year, Month, Day
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day

**Quick Check**

In [ ]:
# See cleaned data
df.head()

# Confirm TotalSales column
df[['InvoiceNo','Quantity','UnitPrice','TotalSales']].head()

,InvoiceNo,Quantity,UnitPrice,TotalSales
0,536365,6,2.55,15.30
1,536365,6,3.39,20.34
2,536365,8,2.75,22.00
3,536365,6,3.39,20.34
4,536365,6,3.39,20.34


**Exploratory Data Analysis (EDA)**

**Overall Sales Summary**

In [ ]:
# Total Revenue
total_revenue = df['TotalSales'].sum()

# Total Orders (unique InvoiceNo)
total_orders = df['InvoiceNo'].nunique()

# Total Customers
total_customers = df['CustomerID'].nunique()

print("Total Revenue:", round(total_revenue, 2))
print("Total Orders:", total_orders)
print("Total Customers:", total_customers)

Total Revenue: 8911407.9
Total Orders: 18536
Total Customers: 4339


**Top 10 Products by Revenue**

In [ ]:
top_products = (df.groupby('Description')['TotalSales']
                  .sum()
                  .sort_values(ascending=False)
                  .head(10))

print(top_products)

Description
PAPER CRAFT , LITTLE BIRDIE           168469.60
REGENCY CAKESTAND 3 TIER              142592.95
WHITE HANGING HEART T-LIGHT HOLDER    100448.15
JUMBO BAG RED RETROSPOT                85220.78
MEDIUM CERAMIC TOP STORAGE JAR         81416.73
POSTAGE                                77803.96
PARTY BUNTING                          68844.33
ASSORTED COLOUR BIRD ORNAMENT          56580.34
Manual                                 53779.93
RABBIT NIGHT LIGHT                     51346.20
Name: TotalSales, dtype: float64


**Top 10 Customers by Revenue**

In [ ]:
top_customers = (df.groupby('CustomerID')['TotalSales']
                   .sum()
                   .sort_values(ascending=False)
                   .head(10))

print(top_customers)

CustomerID
14646.0    280206.02
18102.0    259657.30
17450.0    194550.79
16446.0    168472.50
14911.0    143825.06
12415.0    124914.53
14156.0    117379.63
17511.0     91062.38
16029.0     81024.84
12346.0     77183.60
Name: TotalSales, dtype: float64


**Monthly Revenue Trend**

In [ ]:
monthly_sales = (df.groupby(['Year','Month'])['TotalSales']
                   .sum()
                   .reset_index())

# Create Year-Month column for readability
monthly_sales['YearMonth'] = monthly_sales['Year'].astype(str) + '-' + monthly_sales['Month'].astype(str)

print(monthly_sales.head())

   Year  Month  TotalSales YearMonth
0  2010     12  572713.890   2010-12
1  2011      1  569445.040    2011-1
2  2011      2  447137.350    2011-2
3  2011      3  595500.760    2011-3
4  2011      4  469200.361    2011-4


**Sales by Country**

In [ ]:
country_sales = (df.groupby('Country')['TotalSales']
                   .sum()
                   .sort_values(ascending=False)
                   .head(10))

print(country_sales)

Country
United Kingdom    7308391.554
Netherlands        285446.340
EIRE               265545.900
Germany            228867.140
France             209024.050
Australia          138521.310
Spain               61577.110
Switzerland         56443.950
Belgium             41196.340
Sweden              38378.330
Name: TotalSales, dtype: float64


**Returns Analysis**

In [ ]:
returns_summary = (returns_df.groupby('Description')['Quantity']
                     .sum()
                     .sort_values(ascending=True)
                     .head(10))

print(returns_summary)

Description
PAPER CRAFT , LITTLE BIRDIE           -80995
MEDIUM CERAMIC TOP STORAGE JAR        -74494
ROTATING SILVER ANGELS T-LIGHT HLDR    -9367
Manual                                 -3995
FAIRY CAKE FLANNEL ASSORTED COLOUR     -3150
PINK BLUE FELT CRAFT TRINKET BOX       -2617
WHITE HANGING HEART T-LIGHT HOLDER     -2578
GIN + TONIC DIET METAL SIGN            -2030
HERB MARKER BASIL                      -1527
FELTCRAFT DOLL MOLLY                   -1447
Name: Quantity, dtype: int64


**Export Cleaned Data for Dashboards**

**Export Cleaned Sales Data**

In [ ]:
# Export cleaned sales data
df.to_csv("Cleaned_Ecommerce_Sales.csv", index=False)

print("✅ Cleaned sales data exported successfully!")

✅ Cleaned sales data exported successfully!


**Export Returns Data (Optional)**

In [ ]:
# Export returns data separately
returns_df.to_csv("Ecommerce_Returns.csv", index=False)

print("✅ Returns data exported successfully!")

✅ Returns data exported successfully!


**Download the Files to Your Computer**

In [ ]:
from google.colab import files

# Download sales file
files.download("Cleaned_Ecommerce_Sales.csv")

# Download returns file
files.download("Ecommerce_Returns.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>